# Interval Replacements of Persistence Modules

In this tutorial notebook, we focus on a specific example of (quiver) representation, also called persistence module, to illustrate how to compute its interval replacement using the provided code in the **utils.py** file. Some visualization features from **display.py** are available in 1D or 2D settings. Note that all the computations should be valid for any d-dimensional grid with $d \ge 1$.

In this tutorial you will see how to:
- **define a representation** object ;
- **define intervals** of this representation ;
- **compute interval ranks and interval signed multiplicities**, which yield the interval replacement of the representation.

## Setup: import

All the necessary code to compute interval replacements can be found in the **utils.py** file. The **display.py** file contains some functions to help visualizing representations of 1D or 2D grids.

In [1]:
import sympy as sp
from utils import *
from display import display_rep, display_interval

## Create a grid and define a representation

We will use the 2D grid example from the paper (Example 7.2) as our running example. You are welcome to explore your own quiver or representation.

Note: in what follows we will consider $\mathbb{k} = \mathbb{R}$. If necessary, you can modify the code to compute path evaluations, interval ranks, and interval replacements in finite fields for example.

First, we can create an instance of the *Representation* class as follows, by specifying the dimensions of the underlying quiver.

In [2]:
dimensions = [2, 5]  # 2D grid 2x5
R = Representation(dimensions)

We can now visualize the representation thanks to **display_rep**. 

In [3]:
display_rep(R)

.          .          .          .          .
                                                       
                                                       
                                                       
.          .          .          .          .


At the moment, the representation is a 2x5 grid with no vector space. We can add vector spaces manually.

In [4]:
# manually add vector spaces
R.create_vecs((0, 0), 0) # (position, vector space's dimension)
R.create_vecs((0, 1), 1)
R.create_vecs((0, 2), 2)
R.create_vecs((0, 3), 2)
R.create_vecs((0, 4), 1)
R.create_vecs((1, 0), 1)
R.create_vecs((1, 1), 2)
R.create_vecs((1, 2), 2)
R.create_vecs((1, 3), 1)
R.create_vecs((1, 4), 0)

In [5]:
display_rep(R)

0          1          2          2          1
                                                       
                                                       
                                                       
1          2          2          1          0


Note that, relative to the conventions in the paper, the vertical direction of the quiver arrows is flipped (they point downward rather than upward). This convention is more convenient for computer implementations.

Now that we successfully added all the vector spaces, we can specify linear maps.

In [6]:
# first define parameter lambda

la = 1 # take lambda=1
 
# our implementation also handles symbolic calculations
# try to uncomment the following line to obtain symbolic results that depend on lambda:
# la = sp.symbols('lambda')

# manually add linear maps u: x -> y
R.create_matrix((0, 0), (0, 1), None) # (x, y, matrix), for matrices equal to 0 you can directly write None
R.create_matrix((0, 1), (0, 2), sp.Matrix([[0],[1]]))
R.create_matrix((0, 2), (0, 3), sp.eye(2))
R.create_matrix((0, 3), (0, 4), sp.Matrix([[1,-1]]))
R.create_matrix((1, 0), (1, 1), sp.Matrix([[1],[0]]))
R.create_matrix((1, 1), (1, 2), sp.eye(2))
R.create_matrix((1, 2), (1, 3), sp.Matrix([[la,-1]]))
R.create_matrix((1, 3), (1, 4), None)
R.create_matrix((0, 0), (1, 0), None)
R.create_matrix((0, 1), (1, 1), sp.Matrix([[0],[1]]))
R.create_matrix((0, 2), (1, 2), sp.eye(2))
R.create_matrix((0, 3), (1, 3), sp.Matrix([[la,-1]]))
R.create_matrix((0, 4), (1, 4), None)

In [7]:
display_rep(R)

0 -------> 1 -------> 2 -------> 2 -------> 1
|          |          |          |          |          
|          |          |          |          |          
v          v          v          v          v          
1 -------> 2 -------> 2 -------> 1 -------> 0


Once we have specified all the matrices, we can now evaluate $L(\rho)$, given a path $\rho: x \rightarrow y$.

In [8]:
R.evaluation((0,1), (1,2)) # (x, y)

Matrix([
[0],
[1]])

This evaluation attribute will be useful to compute interval ranks and interval signed multiplicities. But first, we need to define intervals.

## Define intervals

We can define intervals by providing a list of sources and a list of sinks.

In [9]:
itv1 = Interval([(1,1),(0,2)], [(1,3)]) # (sources, sinks)

Let us display the interval.

In [10]:
display_interval(R, itv1)

.          .          X -------> X          .
                      |          |                     
                      |          |                     
                      v          v                     
.          X -------> X -------> X          .


Note that the intervals are defined through their sources and their sinks. If needed, you can access all the points in the interval with the **int_hull** attribute.

In [11]:
R.int_hull(itv1)

[(1, 2), (1, 1), (0, 3), (0, 2), (1, 3)]

Conversely, given a list of points forming a convex and connected set, we can create an Interval object with **get_src_snk**:

In [12]:
points = [(1, 2), (1, 1), (0, 3), (0, 2), (1, 3)]
src, snk =  R.get_src_snk(points)
itv2 = Interval(src,snk)
display_interval(R, itv2)

.          .          X -------> X          .
                      |          |                     
                      |          |                     
                      v          v                     
.          X -------> X -------> X          .


You can access the list of all intervals with **list_int**.

In [13]:
intervals = R.list_int(conv=False)

By **[4, Theorem 31]**, we know that the number of intervals should be equal to $\sum\limits_{w=1}^5 \sum\limits_{h=1}^2 \frac{(3-w+1)(3-h+1)}{h+w-1} \binom{h+w-1}{h-1}\binom{h+w-1}{w-1}$.

In [14]:
import math
m = 2
n = 5
S = 0
for w in range(1,m+1):
    for h in range(1,n+1):
        S += (m-w+1)*(n-h+1)/(h+w-1)*math.comb(h+w-1,h-1)*math.comb(h+w-1,w-1)
print(int(S))

100


The number of intervals is indeed equal to 100 here:

In [15]:
print(len(intervals))

100


## Interval ranks

We can directly compute the $I$-rank of $R$ using **int_rank** as follows.

Note that, by default, the *tot* compression system is used. If you prefer to use the *ss* compression system, you can specify it.

In [16]:
R.int_rank(itv1, compression="ss")

1

## Interval replacement

Now that we can compute interval ranks, we can compute the interval replacement of the representation. Again, we refer to **[1]** for the notations. 

In the implementation, we used the definition using the Möbius inversion.

In [17]:
R.int_replacement(itv1, compression="ss")

0

Let us compute all the interval signed multiplicities and display the nonzero ones.

In [18]:
for itv in intervals:
        
    repl = R.int_replacement(itv, compression='ss')
    if repl != 0:
        print(f"Interval: src={itv.src}, snk={itv.snk}")
        print()
        display_interval(R,itv)
        print()
        print("Interval signed multiplicity:")
        sp.pprint(repl)
        print("="*60)

Interval: src=[(0, 2), (1, 1)], snk=[(0, 3), (1, 2)]

.          .          X -------> X          .
                      |                                
                      |                                
                      v                                
.          X -------> X          .          .

Interval signed multiplicity:
1
Interval: src=[(0, 2), (1, 1)], snk=[(0, 4), (1, 2)]

.          .          X -------> X -------> X
                      |                                
                      |                                
                      v                                
.          X -------> X          .          .

Interval signed multiplicity:
-1
Interval: src=[(0, 1)], snk=[(0, 4), (1, 2)]

.          X -------> X -------> X -------> X
           |          |                                
           |          |                                
           v          v                                
.          X -------> X          .          .

# Important note when using symbolic calculation

When working with symbolic lambda parameters rather than fixed numerical values, we need to ensure that substituting specific numerical values into our symbolic results matches what we would get by building the representation with those numerical values directly.

However, this direct substitution does not always work seamlessly. The interval replacement computation depends on matrix rank calculations, which can behave discontinuously. This means the underlying algebraic structure actually changes at certain special parameter values, making simple substitution unreliable at these singular points.

Our next step is to systematically identify these singular parameter values where the symbolic computation breaks down. For these special cases, we could compute the interval replacements numerically to obtain the correct results.

In [19]:
### uncomment this block when using symbolic calculation (la = sp.symbols('lambda'))

# # find singularities
# singularities = R.find_complete_singularities(la)

# print(f"""
# For λ ∉ {singularities}:
#   - Symbolic interval replacement computations are valid
#   - Simple substitution λ → gives correct results
#   - Algebraic structure remains constant

# For λ ∈ {singularities}:
#   - Algebraic structure changes at these parameter values  
#   - Symbolic computation cannot be trusted via simple substitution
#   - Interval replacements must be computed numerically for each singularity
# """)

## References

**[1]**: Asashiba, H., Gauthier, E., & Liu, E. *Interval Replacements of Persistence Modules*. arXiv preprint [arXiv:2403.08308](https://arxiv.org/abs/2403.08308) (2024).

**[2]**: Asashiba, H., Escolar, E. G., Nakashima, K., & Yoshiwaki, M. *On Approximation of 2D Persistence Modules by Interval-decomposables*. Journal of Computational Algebra, Volumes 6–7, 2023, 100007, ISSN 2772-8277, [https://doi.org/10.1016/j.jaca.2023.100007](https://doi.org/10.1016/j.jaca.2023.100007).

**[3]**: Kim, W., & Mémoli, F. *Generalized persistence diagrams for persistence modules over posets*. J Appl. and Comput. Topology 5, 533–581 (2021). [https://doi.org/10.1007/s41468-021-00075-1](https://doi.org/10.1007/s41468-021-00075-1).

**[4]**: Asashiba, H., Buchet, M., Escolar, E. G., Nakashima, K., & Yoshiwaki, M. *On interval decomposability of 2D persistence modules*, Computational Geometry, Volumes 105–106, 2022, 101879, ISSN 0925-7721, [https://doi.org/10.1016/j.comgeo.2022.101879](https://doi.org/10.1016/j.comgeo.2022.101879).